In [57]:
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads
        
        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.linear_v = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.linear_k = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.linear_q = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        print(f"==== 기본 정보 ====")
        print(f"embedding size는 {self.embed_size} 입니다.")
        print(f"heads 수는 {self.heads} 입니다.")
        print(f"head의 dimension은 {self.head_dim} 입니다.")
        print('\n')
        print(f"==== 초기 정보 ====")
        print(f"values.shape : {values.shape}")
        print(f"keys.shape : {keys.shape}")
        print(f"query.shape : {query.shape}")
        if mask != None:
            print(f"mask.shape : {mask.shape}")
        
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # q,k,v 분할 (head 수로)
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)
        print('\n')
        print(f"==== Multi-Head에 의해 분할합니다 ====")
        print(f"values.shape : {values.shape}")
        print(f"keys.shape : {keys.shape}")
        print(f"query.shape : {queries.shape}")

        values = self.linear_v(values)
        keys = self.linear_k(keys)
        queries = self.linear_q(queries)
        print('\n')
        print("==== 선형레이어를 통과합니다. ====")
        print(f"최종 values.shape : {values.shape}")
        print(f"최종 keys.shape : {keys.shape}")
        print(f"최종 query.shape : {queries.shape}")

        # Attention 수행 (최종 shape 형태 : [batch, head, query, key])
        print('\n')
        print("==== Attention 연산을 수행합니다. ====")
        attention_score = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        print(f"attention_score shape: {attention_score.shape}")
        if mask is not None:
            attention_score = attention_score.masked_fill(mask == 0, float("-1e20"))
            
        attention_distribution = torch.softmax(attention_score / (self.embed_size ** (1/2)), dim=3)
        print(f"attention_distribution shape: {attention_distribution.shape}")

        attention_value_matrix = torch.einsum("nhql,nlhd->nqhd", [attention_distribution, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        print(f"attention_value_matrix shape : {attention_value_matrix.shape}")
        
        out = self.fc_out(attention_value_matrix)
        print(f"out shape : {out.shape}")
        return out


In [58]:
# MHA 레이어 객체 지정
embed_size = 256
heads = 8
attention = MultiHeadAttention(embed_size, heads)

In [66]:
# 더미 입력 데이터 생성
N, seq_length = 1, 4  # 배치 크기와 시퀀스 길이
dummy_values = torch.rand((N, seq_length, embed_size))
dummy_keys = torch.rand((N, seq_length, embed_size))
dummy_query = torch.rand((N, seq_length, embed_size))
dummy_mask = None  # 필요한 경우 마스크를 사용할 수 있습니다.

# 멀티 헤드 주의 레이어 실행
output = attention(dummy_values, dummy_keys, dummy_query, dummy_mask)

==== 기본 정보 ====
embedding size는 256 입니다.
heads 수는 8 입니다.
head의 dimension은 32 입니다.


==== 초기 정보 ====
values.shape : torch.Size([1, 4, 256])
keys.shape : torch.Size([1, 4, 256])
query.shape : torch.Size([1, 4, 256])


==== Multi-Head에 의해 분할합니다 ====
values.shape : torch.Size([1, 4, 8, 32])
keys.shape : torch.Size([1, 4, 8, 32])
query.shape : torch.Size([1, 4, 8, 32])


==== 선형레이어를 통과합니다. ====
최종 values.shape : torch.Size([1, 4, 8, 32])
최종 keys.shape : torch.Size([1, 4, 8, 32])
최종 query.shape : torch.Size([1, 4, 8, 32])


==== Attention 연산을 수행합니다. ====
attention_score shape: torch.Size([1, 8, 4, 4])
attention_distribution shape: torch.Size([1, 8, 4, 4])
attention_value_matrix shape : torch.Size([1, 4, 256])
out shape : torch.Size([1, 4, 256])


In [69]:
print(output.shape)  # 출력의 차원 확인
print(output)

torch.Size([1, 4, 256])
tensor([[[-0.0977, -0.0988, -0.2643,  ...,  0.1221,  0.0456, -0.4240],
         [-0.0977, -0.0985, -0.2649,  ...,  0.1224,  0.0457, -0.4237],
         [-0.0971, -0.0985, -0.2653,  ...,  0.1220,  0.0469, -0.4233],
         [-0.0978, -0.0987, -0.2644,  ...,  0.1225,  0.0456, -0.4239]]],
       grad_fn=<ViewBackward0>)
